<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/Compatibilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Instalación de dependencias
!pip install -q kaggle
!pip install -q scikit-learn
!pip install -q pandas
!pip install -q numpy
!pip install -q tensorflow
!pip install -q matplotlib
!pip install -q seaborn

# 2. Configuración de la API de Kaggle
# Necesitas subir tu archivo kaggle.json que puedes obtener desde tu cuenta de Kaggle
from google.colab import files
import os

# Subir el archivo kaggle.json
print("Por favor, sube tu archivo kaggle.json para acceder a los datos de Kaggle.")
files.upload()

# Crear la carpeta de Kaggle y mover el archivo
os.makedirs('/root/.kaggle/', exist_ok=True)
!mv kaggle.json /root/.kaggle/
os.chmod('/root/.kaggle/kaggle.json', 600)

# 3. Descargar el conjunto de datos desde Kaggle
# Reemplaza 'username/dataset-name' con el identificador de tu conjunto de datos
# Por ejemplo: 'zynicide/wine-reviews'
# Asegúrate de que el conjunto de datos contiene la información necesaria para las habitaciones y residentes

# Ejemplo:
# !kaggle datasets download -d username/dataset-name

# Para este ejemplo, supongamos que el dataset se llama 'roommate-compatibility'
# !kaggle datasets download -d yourusername/roommate-compatibility

# Descomprimir el archivo descargado
# !unzip roommate-compatibility.zip

# Nota: Asegúrate de reemplazar con el nombre correcto del archivo

# 4. Cargar y preprocesar los datos
import pandas as pd
import numpy as np

# Cargar el dataset
# Supongamos que el archivo es 'roommate_data.csv'
# Asegúrate de que el archivo esté en el directorio de trabajo
# Si está en otra carpeta, proporciona la ruta correcta

try:
    data = pd.read_csv('roommate_data.csv')
except FileNotFoundError:
    print("El archivo 'roommate_data.csv' no se encontró. Por favor, verifica el nombre y la ubicación del archivo.")

# Exploración básica de los datos
print("Primeras filas del dataset:")
print(data.head())

print("\nInformación del dataset:")
print(data.info())

print("\nDescripción estadística:")
print(data.describe())

# Manejo de valores faltantes
print("\nValores faltantes por columna:")
print(data.isnull().sum())

# Opcional: Puedes decidir cómo manejar los valores faltantes
# Por ejemplo, eliminar filas con valores faltantes
data = data.dropna()

# Codificación de variables categóricas
from sklearn.preprocessing import LabelEncoder

# Identifica las columnas categóricas
categorical_cols = ['gusta_noche', 'gusta_dia', 'vida_social', 'trabaja_remoto']  # Añade más según tu dataset

le = LabelEncoder()
for col in categorical_cols:
    data[col] = le.fit_transform(data[col])

# Separar características y etiqueta
# Supongamos que la etiqueta es 'compatibilidad' (1 compatible, 0 no compatible)
X = data.drop('compatibilidad', axis=1)
y = data['compatibilidad']

# División en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalado de características
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. Entrenamiento del modelo
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Definir el modelo
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nPrecisión en el conjunto de prueba: {accuracy*100:.2f}%")

# 6. Visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Pérdida durante el entrenamiento
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Pérdida de Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de Validación')
plt.title('Pérdida del Modelo')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# Precisión durante el entrenamiento
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='Precisión de Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de Validación')
plt.title('Precisión del Modelo')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()

plt.show()

# 7. Función de recomendación
def recomendar_habitacion(inputs):
    """
    Recibe un diccionario con las características del usuario y recomienda la compatibilidad.

    Parámetros:
    inputs (dict): Diccionario con las siguientes claves:
        - 'gusta_noche': 'si' o 'no'
        - 'gusta_dia': 'si' o 'no'
        - 'vida_social': 'alta', 'media', 'baja'
        - 'trabaja_remoto': 'si' o 'no'
        # Añade más características según tu formulario

    Retorna:
    compatibilidad (float): Probabilidad de compatibilidad.
    """
    # Convertir las entradas en un DataFrame
    input_df = pd.DataFrame([inputs])

    # Codificar las variables categóricas
    for col in categorical_cols:
        input_df[col] = le.transform(input_df[col])

    # Escalar las características
    input_scaled = scaler.transform(input_df)

    # Hacer la predicción
    compatibilidad = model.predict(input_scaled)[0][0]

    return compatibilidad

# 8. Ejemplo de uso de la función de recomendación
# Definir las entradas del usuario
entrada_usuario = {
    'gusta_noche': 'si',
    'gusta_dia': 'no',
    'vida_social': 'alta',
    'trabaja_remoto': 'si'
    # Añade más características según tu formulario
}

# Obtener la compatibilidad
compatibilidad = recomendar_habitacion(entrada_usuario)
print(f"\nProbabilidad de compatibilidad: {compatibilidad*100:.2f}%")

# Interpretación de la compatibilidad
if compatibilidad >= 0.7:
    print("Alta compatibilidad. Se recomienda esta habitación.")
elif compatibilidad >= 0.4:
    print("Compatibilidad media. Requiere revisión adicional.")
else:
    print("Baja compatibilidad. Se desaconseja esta habitación.")